# chap 2. sklearn으로 시작하는 ML

## 교차검증 - K-fold와 Stratified K-fold

* 교차검증 - 필수적 요소

### k-fold 교차검증

- 가장 보편적인 사용
- k개의 데이터 폴드 세트를 만들어 k번만큼 각 폴트 세트에 학습과 검증 평가를 반복적으로 수행

In [3]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np

iris = load_iris()
features = iris.data
label = iris.target
dt_clf = DecisionTreeClassifier(random_state=156)

# 5개의 폴드 세트로 분리하는 kfold 객체와 폴드 세트별 정확도를 담을 리스트 객체 생성
kfold = KFold(n_splits=5)
cv_accuracy = []
print('붓꽃 데이터 셋 크기:', features.shape[0])

붓꽃 데이터 셋 크기: 150


In [5]:
n_iter = 0

# k-fold 객체의 split()을 호출하면 폴드 별 학습용, 검증요 테스트이 row 인덱스를 array로 변환
for train_index, test_index in kfold.split(features): # 자동으로 5번 실행시켜주는 구문
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)
    n_iter += 1
    
    accuracy = np.round(accuracy_score(y_test,pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print('\n#{0} 교차 검증 정확도 : {1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'
         .format(n_iter, accuracy, train_size, test_size))
    print('\n{0} 검증 세트 인덱스:{1}'.format(n_iter, test_index))
    
    cv_accuracy.append(accuracy)
    
print('\n## 평균 검증 정확도:', np.mean(cv_accuracy))


#1 교차 검증 정확도 : 1.0, 학습 데이터 크기: 120, 검증 데이터 크기: 30

1 검증 세트 인덱스:[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]

#2 교차 검증 정확도 : 0.9667, 학습 데이터 크기: 120, 검증 데이터 크기: 30

2 검증 세트 인덱스:[30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]

#3 교차 검증 정확도 : 0.8667, 학습 데이터 크기: 120, 검증 데이터 크기: 30

3 검증 세트 인덱스:[60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]

#4 교차 검증 정확도 : 0.9333, 학습 데이터 크기: 120, 검증 데이터 크기: 30

4 검증 세트 인덱스:[ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]

#5 교차 검증 정확도 : 0.7333, 학습 데이터 크기: 120, 검증 데이터 크기: 30

5 검증 세트 인덱스:[120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]

## 평균 검증 정확도: 0.9


### stratified k-fold 교차 검증

- 불균형한 분포도를 가진 label 데이터 집합을 위한 방식
- 학습 데이터와 검증 데이터 셋이 가지는 label 분포도가 유사하도록 검증 데이터 추출

In [10]:
import pandas as pd

iris = load_iris()
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['label'] = iris.target
iris_df['label'].value_counts()

2    50
1    50
0    50
Name: label, dtype: int64

In [20]:
# k-fold: 이렇게 하면 안됌
kfold = KFold(n_splits=3)

n_iter = 0
for train_index, test_index in kfold.split(iris_df):
    n_iter +=1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print('\n<< {0}번째 교차 검증 >>'.format(n_iter))
    print(' 학습 레이블 데이터 분포:\n', label_train.value_counts())
    print(' 검증 레이블 데이터 분포:\n', label_test.value_counts())


<< 1번째 교차 검증 >>
 학습 레이블 데이터 분포:
 2    50
1    50
Name: label, dtype: int64
 검증 레이블 데이터 분포:
 0    50
Name: label, dtype: int64

<< 2번째 교차 검증 >>
 학습 레이블 데이터 분포:
 2    50
0    50
Name: label, dtype: int64
 검증 레이블 데이터 분포:
 1    50
Name: label, dtype: int64

<< 3번째 교차 검증 >>
 학습 레이블 데이터 분포:
 1    50
0    50
Name: label, dtype: int64
 검증 레이블 데이터 분포:
 2    50
Name: label, dtype: int64


In [23]:
# straitified kfold
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=3)
n_iter = 0

for train_index, test_index in skf.split(iris_df, iris_df['label']):
    n_iter +=1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print('\n<< {0}번째 교차 검증 >>'.format(n_iter))
    print(' 학습 레이블 데이터 분포:\n', label_train.value_counts())
    print(' 검증 레이블 데이터 분포:\n', label_test.value_counts())


<< 1번째 교차 검증 >>
 학습 레이블 데이터 분포:
 2    34
1    33
0    33
Name: label, dtype: int64
 검증 레이블 데이터 분포:
 1    17
0    17
2    16
Name: label, dtype: int64

<< 2번째 교차 검증 >>
 학습 레이블 데이터 분포:
 1    34
2    33
0    33
Name: label, dtype: int64
 검증 레이블 데이터 분포:
 2    17
0    17
1    16
Name: label, dtype: int64

<< 3번째 교차 검증 >>
 학습 레이블 데이터 분포:
 0    34
2    33
1    33
Name: label, dtype: int64
 검증 레이블 데이터 분포:
 2    17
1    17
0    16
Name: label, dtype: int64


In [26]:
dt_clf = DecisionTreeClassifier(random_state=156)

skfold = StratifiedKFold(n_splits=3)
n_iter = 0
cv_accuracy = []

for train_index, test_index in skfold.split(features, label):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)
    
    n_iter += 1
    accuarcy = np.round(accuracy_score(y_test, pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    
    print('\n#{0} 교차 검증 정확도 : {1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'
         .format(n_iter, accuracy, train_size, test_size))
    print('{0} 검증 세트 인덱스:{1}'.format(n_iter, test_index))
    
    cv_accuracy.append(accuracy)
    
print('\n## 교차 검증별 정확도:', np.round(cv_accuracy, 4))
print('## 평균 검증 정확도:', np.mean(cv_accuracy))
    


#1 교차 검증 정확도 : 0.7333, 학습 데이터 크기: 100, 검증 데이터 크기: 50
1 검증 세트 인덱스:[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  50
  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115]

#2 교차 검증 정확도 : 0.7333, 학습 데이터 크기: 100, 검증 데이터 크기: 50
2 검증 세트 인덱스:[ 17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82 116 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132]

#3 교차 검증 정확도 : 0.7333, 학습 데이터 크기: 100, 검증 데이터 크기: 50
3 검증 세트 인덱스:[ 34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  83  84
  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149]

## 교차 검증별 정확도: [0.7333 0.7333 0.7333]
## 평균 검증 정확도: 0.7333


### cross_val_score()

- 교차 검증을 보다 간편하게
- 폴드 세트 추출, 학습, 예측, 평가를 한번에 수행

In [27]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.datasets import load_iris

iris_data = load_iris()
dt_clf = DecisionTreeClassifier(random_state=156)

data = iris_data.data
label = iris_data.target

# 성능 지표는 정확도, 교차 검증 세트는 3개
scores = cross_val_score(dt_clf, data, label, scoring='accuracy', cv=3) # scoring: 평가를 뭘로할래?
print('교차 검증별 정확도:', np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores), 4))

교차 검증별 정확도: [0.98 0.94 0.98]
평균 검증 정확도: 0.9667


### GridSerchCV - 교차 검증과 최적 하이퍼 파라미터 튜닝을 한번에

In [28]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score

iris = load_iris()

X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target,
                                                   test_size=0.2, random_state=121)

dtree = DecisionTreeClassifier()

parameters = {'max_depth': [1, 2, 3], 'min_samples_split': [2 ,3]}

In [29]:
import pandas as pd

# param_grid의 하이퍼 파라미터들을 3개의 train, test set fold로 나누어서 테스트 수행 결정
## refit=Truerk default임. True이면 가장 좋은 파라미터 설정으로 재학습시킴.
grid_dtree = GridSearchCV(dtree, param_grid=parameters, cv=3, refit=True, return_train_score=True)

# 붓꽃 train데이터로 학습
grid_dtree.fit(X_train, y_train)

# 결과는 cv_results_ 의 dictionary에 저장됌
scores_df = pd.DataFrame(grid_dtree.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score',
          'split0_test_score', 'split1_test_score', 'split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5,0.700,0.7,0.70
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5,0.700,0.7,0.70
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3,0.925,1.0,0.95
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3,0.925,1.0,0.95
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1,0.975,1.0,0.95
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1,0.975,1.0,0.95


In [30]:
grid_dtree.cv_results_

{'mean_fit_time': array([0.00122746, 0.00066582, 0.00051562, 0.00040825, 0.00036073,
        0.00035373]),
 'std_fit_time': array([4.70291040e-04, 4.05799037e-05, 9.65098330e-05, 1.32507737e-06,
        8.84401178e-06, 2.92865845e-06]),
 'mean_score_time': array([0.00056887, 0.00051991, 0.00025439, 0.00023301, 0.00020194,
        0.00020162]),
 'std_score_time': array([1.21798529e-04, 1.39101980e-04, 1.37706080e-05, 3.02414856e-06,
        7.78671819e-07, 2.92865845e-06]),
 'param_max_depth': masked_array(data=[1, 1, 2, 2, 3, 3],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_min_samples_split': masked_array(data=[2, 3, 2, 3, 2, 3],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 1, 'min_samples_split': 2},
  {'max_depth': 1, 'min_samples_split': 3},
  {'max_depth': 2, 'min_samples_split': 2},
  {'max_depth': 2, 'min_sa

In [31]:
print('GV 최적의 파라미터:', grid_dtree.best_params_)
print('GV 최고 정확도:{0:.4f}'.format(grid_dtree.best_score_))

pred = grid_dtree.predict(X_test)
print('test dataset 정확도:{0:.4f}'.format(accuracy_score(y_test, pred)))

GV 최적의 파라미터: {'max_depth': 3, 'min_samples_split': 2}
GV 최고 정확도:0.9750
test dataset 정확도:0.9667


In [32]:
estimator = grid_dtree.best_estimator_

pred = estimator.predict(X_test)
print('test dataset 정확도:{0:.4f}'.format(accuracy_score(y_test, pred)))

test dataset 정확도:0.9667
